# Test the functions in Project 1

This is part of one of three mandatory projects of the course FYS-STK4155 (University of Oslo). In this file, we test the performance of our own functions by comparing them with the functions in the scikit learn python package.

The Franke's function is given for this project, so we are not going to test it.

In [83]:
def franke_function(x, y):
    term1 = 0.75 * np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75 * np.exp(-((9*x+1)**2) / 49.0 - 0.1*(9*y+1))
    term3 = 0.5 * np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2 * np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4

Import packages

In [84]:
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.linear_model import LinearRegression 
from sklearn.linear_model import Ridge
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import algorithms
import model_selection0
from model_comparison0 import model_comparison0
from utils import generateDesignmatrix, mean_squared_error, r2_score

#  1) generateDesignmatrix in functions.py

This functions generates a design matrix for the arrays x and y, and a given polynonial order p. The arrays x and y are the ones that define the meshgrid for the calculation of z with the Franke function.

- We define the vectors x and y, and apply the Franke's function to these.
- m defines the size of the meshgrid. Here we choose a small number so that the calculation can easily checked by hand.
- x, y, and z are arrays. The length of x and y is m, which determines the dimensions of z.

In [85]:
np.random.seed(1000)
m = 5  
x = np.random.rand(m, )
y = np.random.rand(m, )
z = franke_function(x, y)



In [86]:
print('array x:', x)
print('array y:', y)
print('array z:', z)

array x: [0.65358959 0.11500694 0.95028286 0.4821914  0.87247454]
array y: [0.21233268 0.04070962 0.39719446 0.2331322  0.84174072]
array z: [0.50251014 0.90614694 0.32573166 0.56740664 0.0665042 ]


### Test the designMatrix for p=0. 
- When p=0 we expect the designMatrix functions to generate a matrix X which only consist of the terms that correspond to the intercept.
- This means that X has only a column of length equal to the length of x and y, and all its elements are 1.

In [87]:
p = 0 # [1:5]
X = generateDesignmatrix(p,x,y)
print('Design matrix:', X) 

Design matrix: [[1.]
 [1.]
 [1.]
 [1.]
 [1.]]


### Test the designMatrix for p=1. 
- When p=1 we expect the designMatrix functions to generate a matrix X with three columns, the first filled with ones, and the others with the vextors x and y.
- As we have seen before, and the second and third with the values of x and y.

In [88]:

p = 1 # [1:5]
X = generateDesignmatrix(p,x,y)
print('Design matrix:', X) 

Design matrix: [[1.         0.65358959 0.21233268]
 [1.         0.11500694 0.04070962]
 [1.         0.95028286 0.39719446]
 [1.         0.4821914  0.2331322 ]
 [1.         0.87247454 0.84174072]]


More general, the columns of the design matrix will be:
[ones, x, y, x^2, y^2, xy,..., x^py^0,..., x^(p-i)y^i, ..., x^0y^p].

Thus, for a given p, we have t = (p^2+3p+2)/2 number of columns in the design matrix X.

### Let's try with a higher order poynomial (p=3).

In [89]:
p = 3

X = generateDesignmatrix(p,x,y)
print('Design matrix:', X) 


Design matrix: [[1.00000000e+00 6.53589585e-01 2.12332681e-01 4.27179346e-01
  1.38778429e-01 4.50851674e-02 2.79199972e-01 9.07041358e-02
  2.94671959e-02 9.57305446e-03]
 [1.00000000e+00 1.15006943e-01 4.07096248e-02 1.32265970e-02
  4.68188950e-03 1.65727355e-03 1.52115049e-03 5.38449799e-04
  1.90597965e-04 6.74669843e-05]
 [1.00000000e+00 9.50282864e-01 3.97194461e-01 9.03037522e-01
  3.77447090e-01 1.57763440e-01 8.58141083e-01 3.58681502e-01
  1.49919894e-01 6.26627646e-02]
 [1.00000000e+00 4.82191401e-01 2.33132197e-01 2.32508548e-01
  1.12414341e-01 5.43506214e-02 1.12113622e-01 5.42052286e-02
  2.62074023e-02 1.26708798e-02]
 [1.00000000e+00 8.72474535e-01 8.41740724e-01 7.61211815e-01
  7.34397347e-01 7.08527447e-01 6.64137924e-01 6.40742984e-01
  6.18172155e-01 5.96396406e-01]]


- As expected, X has 10 columns. t = (3^3 + 3*3 + 2)/2 = 10
- The first column consists of ones, the second and the third are the vectors x and y.

### Check wether the column number 9 is equal to x*y^2

In [90]:
y2 = y**2
col9 = X[:, 8]

print('Has the ninth column in X the values we expect?:', col9==x*y2)

Has the ninth column in X the values we expect?: [ True  True  True  True  True]


# 2) OLS

We test our own code for the OLS comparing it with the scikit-learn function
When running OLS, x cannot be equal to y since that generates a singular design matrix, and our code will not run.

In [91]:
x = np.linspace(1, 10, num=10)
y = np.linspace(1.1, 10.1, num=10)
z = franke_function(x, y)
p = 2
X = generateDesignmatrix(p,x,y)

In [92]:
our_ols = algorithms.OLS()
our_ols.fit(X,z)
our_betas = our_ols.coef_
print('Regression coefficients for the OLS calculated with our functions:', our_betas)

Regression coefficients for the OLS calculated with our functions: [ 0.02929223 -0.01249085  0.00209199 -0.00138651  0.00031428  0.00180701]


In [93]:
# Create linear regression object
scikit_ols = LinearRegression(fit_intercept=False)

# Train the model using the training sets
scikit_ols.fit(X, z)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=1, normalize=False)

In [94]:
# get the coefficients (?)
scikit_betas = scikit_ols.coef_[0:9]
print('Regression coefficients for the OLS calculated with scikit-learn:', scikit_betas)

Regression coefficients for the OLS calculated with scikit-learn: [ 0.02986962 -0.00647389 -0.00348693  0.00079619  0.0001488  -0.00019989]


In the example above, all the coefficients are close to zero, both with our function and with the scikit.learn function and we notice that the signs of some of the coefficients are different. The intercept is the same.

Therefore, we try with random selection of x and y and some larger samples

In [95]:
np.random.seed(1000)
m = 30  # m defines the size of the meshgrid.
x = np.random.rand(m, )
y = np.random.rand(m, )
z = franke_function(x, y)
p = 2
X = generateDesignmatrix(p,x,y)

In [96]:
our_ols = algorithms.OLS()
our_ols.fit(X,z)
our_betas = our_ols.coef_
print('Regression coefficients for the OLS calculated with our functions:', our_betas)

Regression coefficients for the OLS calculated with our functions: [ 1.15707568 -1.22131811 -0.72666861  0.24054396  1.13194101 -0.43215975]


In [97]:
print('OLS prediction with our functions:', X@our_betas)

OLS prediction with our functions: [0.13468539 0.12713231 0.21728842 0.6082088  0.31257862 0.83608651
 0.4222509  0.53106781 0.13968897 0.32624696 0.62073605 0.38480847
 0.31299295 0.60272371 0.38959959 0.15241299 0.15293553 0.23529875
 0.24239559 0.49563065 0.54789871 0.26750687 0.33570208 0.41054777
 0.46519288 0.587262   0.3109471  0.45247379 0.37315466 0.53325455]


In [98]:
# Create linear regression object
scikit_ols = LinearRegression(fit_intercept=False)

# Train the model using the training sets
scikit_ols.fit(X, z)

#LinearRegression(fit_intercept=True)
# get the coefficients
scikit_betas = scikit_ols.coef_[:]
print('Regression coefficients for the OLS calculated with scikit-learn:', scikit_betas)

Regression coefficients for the OLS calculated with scikit-learn: [ 1.15707568 -1.22131811 -0.72666861  0.24054396  1.13194101 -0.43215975]


In [99]:
print('Difference of the coefficients for the OLS:', our_betas - scikit_betas)

Difference of the coefficients for the OLS: [ 6.88338275e-15 -3.06421555e-14  1.69864123e-14  2.38142839e-14
  1.84297022e-14 -1.56541446e-14]


In this case, te difference between the coefficients calculated with our functions and with the scikit-learn library is of order 10^-14

# 3) Ridge

In [100]:
our_ridge = algorithms.Ridge(lmd = 0.1)
our_ridge.fit(X,z)
our_betas = our_ridge.coef_
print('Regression coefficients for the ridge calculated with our functions:', our_betas)

Regression coefficients for the ridge calculated with our functions: [ 0.7771756  -0.31589716 -0.16606758 -0.25684846  0.4192761  -0.5259916 ]


In [101]:
# X@our_betas

In [102]:
from sklearn import linear_model
scikit_ridge=linear_model.RidgeCV(alphas=[0.1], fit_intercept=False)
scikit_ridge.fit(X, z)

#LinearRegression(fit_intercept=True)
# get the coefficients
scikit_betas = scikit_ridge.coef_[:]
print('Regression coefficients for the OLS calculated with scikit-learn:', scikit_betas)

Regression coefficients for the OLS calculated with scikit-learn: [ 0.7771756  -0.31589716 -0.16606758 -0.25684846  0.4192761  -0.5259916 ]


In [103]:
print('Difference of the coefficients for the ridge:', our_betas - scikit_betas)

Difference of the coefficients for the ridge: [5.22915045e-14 9.49240686e-15 2.45359288e-14 2.16493490e-14
 8.38218384e-15 4.77395901e-15]


# 4) MSE

In [104]:
# our code
print('MSE', mean_squared_error(x, y))

MSE 0.30508884042905127


In [105]:
# our code
our_mse = mean_squared_error(x, y)
# with scikit-learn
scikit_mse = metrics.mean_squared_error(x,y)

print(our_mse - scikit_mse)

0.0


# 5) R squared

In [106]:
# our code
print('R2 score', r2_score(x, y))

R2 score -2.0267315179362133


In [107]:
# our code
our_r2 = r2_score(x, y)
# with scikit-learn
scikit_r2 = metrics.r2_score(x, y)
print(our_r2 - scikit_r2)

0.0
